In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
}

def get_info(url):
    web_data = requests.get(url, headers=headers)
    soup = BeautifulSoup(web_data.text, 'lxml')

    ranks = soup.select('span.pc_temp_num')
    titles = soup.select('div.pc_temp_songlist > ul > li > a')
    times = soup.select('span.pc_temp_tips_r > span')
    
    data_list = []
    for rank, title, time in zip(ranks, titles, times):
        data = {
            "排名": rank.get_text().strip(),
            "歌手": title.get_text().replace("\n", "").replace("\t", "").split('-')[1],
            "歌曲": title.get_text().replace("\n", "").replace("\t", "").split('-')[0],
            "时长": time.get_text().strip()
        }
        data_list.append(data)
    
    return data_list

if __name__ == '__main__':
    urls = ["https://www.kugou.com/yy/rank/home/{}-8888.html".format(str(i)) for i in range(1, 24)]
    all_data = []
    for url in urls:
        data = get_info(url)
        all_data.extend(data)
        time.sleep(1)
    df = pd.DataFrame(all_data)
    df.to_excel('酷狗音乐Top500排行榜.xlsx', index=False)
